In [ ]:
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from PIL import Image
from scipy.integrate import quad
import time

In [ ]:
# Constants and Parameters

# Lengths and elements
L_up = 375  # Length of the inner part of core in cm
plenum = 50  # Length of plenum in cm
L_down = 375 # Length of outer part of core in cm
L_he = 1250  # Length of out of core part in cm
e1 = 2 # Thickness of graphite around a inner fuel channel in cm
e2 = 2 # Thickness of graphite around a outer fuel channel in cm
dz = 5  # Length of each volume element in the z direction in cm
dr = 0.1 # Length of each volume element in the r direction in cm

total_length = L_up + plenum + L_down + L_he
n_elements = int(total_length / dz)
n_up = int((L_up) / dz)
n_down = int((L_down) / dz)
n_plenum = int((plenum) / dz)
r_elements = int((e1 / dr))
core_elements = n_up + n_plenum + n_down
core_length = L_up + plenum + L_down

# Heat exchanger characteristics
# OL = 259370  # Heat transfer area
L = 850 # Length of heat exchanger in cm
OL = 2*np.pi*7.5*L
LL = L_he - L # Length of out of core region minus the heat exchanger

# Material characteristics
rho_fuel = 3241.67e-6   # Density of fuel in kg/cm^3 3155e-6
cp_fuel = 2080.22  # Specific heat of fuel in J/kgK
rho_graph = 1900e-6  # Density of graphite in kg/cm^3
cp_graph = 1760  # Specific heat of graphite in J/kgK
rho_he = 1948.3e-6
cp_he = 1507
rho_cp_fuel = rho_fuel * cp_fuel
rho_cp_graph = rho_graph * cp_graph
rho_cp_he = rho_he * cp_graph

beta_eff_groups = np.array([19, 73.8, 60.5, 71.4, 14.4, 8.9]) * 1e-5
lambda_groups = np.array([0.01264, 0.03293, 0.13511, 0.325, 1.1278, 2.5421])
total_beta_eff = np.sum(beta_eff_groups)

Lambda = 3.3e-4  # Neutron generation time in s
thermal_cond = 0.3 # Thermal conductivity of the graphite in W / cm K

T_0_s = np.array([[900], [850], [850], [850], [850], [850], [850]]) 
T_0_g = np.array([[np.linspace(1000, 1200, r_elements-1)], [np.linspace(950, 1150, r_elements-1)], [np.linspace(950, 1150, r_elements-1)], [np.linspace(950, 1150, r_elements-1)], [np.linspace(950, 1150, r_elements-1)], [np.linspace(950, 1150, r_elements-1)], [np.linspace(950, 1150, r_elements-1)]])
T_ref = np.array([934.9873990764459,870.793839061948,870.793839061948,870.793839061948,870.793839061948,870.793839061948,870.793839061948])
alpha = np.array([-1.8275e-5, -7.4e-6, -7.4e-6, -7.4e-6, -7.4e-6, -7.4e-6, -7.4e-6])
rho_rod = 0

# Flux shape
adjoint_flux_flat = np.zeros(n_elements)
adjoint_flux_flat[:(n_up + n_plenum + n_down)] = 1

flux1 = np.load('fission1.npy')
flux_inverted = flux1[::-1]
flux1 = np.concatenate([flux1, flux_inverted, np.zeros(int(L_he/dz))])

flux2 = np.load('fission2.npy')
flux_inverted = flux2[::-1]
flux2 = np.concatenate([flux2, flux_inverted, np.zeros(int(L_he/dz))])

flux_ratio = np.sum(flux1)/np.sum(flux2)

flux1 /= np.sum(flux1)
flux2 /= np.sum(flux2)

flux3 = flux2
flux4 = flux2
flux5 = flux2
flux6 = flux2
flux7 = flux2

flux = np.vstack((flux1, flux2, flux3, flux4, flux5, flux6, flux7))

In [ ]:
def frac(t):
    frac = np.zeros(n_elements)
    frac[:n_up] = 0.05
    frac[(n_up+n_plenum):(n_up + n_plenum + n_down)] = 0.05
    return frac

def g_zt(t):
    return 100000 # cm^3/s

def radius(n_elements): # Radius difference for up and down channels
    radius = np.ones(n_elements)
    radius[:n_up] = 8
    radius[(n_up + n_plenum):(n_up + n_plenum + n_down)] = 0.5 * 8
    return radius

def graphite_present(i):
    if 0 <= i < n_up or (n_up + n_plenum) <= i < (n_up + n_plenum + n_down):
        return True
    else:
        return False

# Cross-sectional area function
def channels(n_elements):
    channels = np.zeros(n_elements)
    channels[:n_up] = 6
    channels[n_up:(n_up + n_plenum)] = 1
    channels[(n_up+n_plenum):(n_up + n_plenum + n_down)] = 24
    channels[(n_up + n_plenum + n_down):] = 1
    return channels
    
def surface_salt(i):
    surface = np.zeros(n_elements)
    surface[:n_up] = channel_values[1] *radius_values[1]**2*np.pi
    surface[n_up:(n_up + n_plenum)] = 100*np.pi
    surface[(n_up+n_plenum):(n_up + n_plenum + n_down)] = channel_values[n_up+n_plenum+2] * radius_values[n_up+n_plenum+2]**2 * np.pi
    surface[(n_up + n_plenum + n_down):] = OL/L
    return surface

def volume(i, j):
    r_ = r(i) - dr/2
    volume = np.array([np.pi * dz * (r_[idx+1]**2 - r_[idx]**2) for idx in j])
    return volume

def total_volume(i):
    return np.pi * ((r(i)[-1]-dr/2)**2 - (r(i)[0]-dr/2)**2) * dz

def surface(i, j):
    r_ = r(i) - dr/2
    surface = np.array([0 if idx == (r_elements-1) else 2*np.pi * r_[idx] * dz for idx in j])
    return surface

def total_surface(i):
    return total_volume(i)/dz
    
def r(i):
    if i <= n_up:
        return np.linspace(radius_values[0], radius_values[0] + e1, r_elements) + dr/2
    if (n_up + n_plenum) <= i <= (n_up + n_plenum + n_down):
        return np.linspace(radius_values[(n_up + n_plenum + 1)], radius_values[(n_up + n_plenum + 1)] + e2, r_elements) + dr/2
    else:
        return np.zeros(r_elements)

# Heat exchanger functions
def heat_ex(n_elements):
    heat_ex = np.zeros(n_elements)
    heat_ex[int(-(LL/2+L)/dz):int(-(LL/2)/dz)] = 5
    return heat_ex

def O(n_elements):
    O = np.zeros(n_elements)
    O[int(-(LL/2+L)/dz):int(-(LL/2)/dz)] = OL / L
    return O

def T_he(n_elements):
    T_he = np.zeros(n_elements)
    T_he[int(-(LL/2+L)/dz):int(-(LL/2)/dz)] = 700
    return T_he

# Matrix elements
def a_j_precursor(i, group):
    return 1 / dt + lambda_groups[group] + g_zt_values / (surface_salt_values[i] * dz)

def b_j_precursor(i):
    return - g_zt_values / (surface_salt_values[i] * dz)

def B_j_precursor(group, t, n):
    return beta_eff_groups[group] / Lambda * flux[n] / (surface_salt_values * dz) * power[t] * power_frac[n] + precursors[t,n, :, group] / dt

def a_j_temp_s(i):
    return surface_salt_values[i] / dt + g_zt_values / dz + heat_ex_values[i] * circumference[i] / rho_cp_fuel + channel_values[i] * 2*np.pi*radius_values[i]* heat_coef_values[i] / (rho_cp_fuel)

def b_j_temp_s(i):
    return - g_zt_values / dz

def B_j_s(t, n):
    return (1 - frac(t)) * 1 / (rho_cp_fuel) * flux[n] / dz * power[t] * power_frac[n] + T_he_values * heat_ex_values * circumference / (rho_cp_fuel) + channel_values * 2*np.pi*radius_values* heat_coef_values * T_graph[t,n,:,0] / (rho_cp_fuel) + surface_salt_values / dt * T_salt[t,n,:]

def a_j_temp_g(i, j):
    condition = (j == 0)
    
    term1 = rho_cp_graph * (volume(i, j)) / dt
    term2 = (surface(i, j + 1) * thermal_cond / dr + heat_coef_values[i] * surface(i, j))

    return np.where(condition, term1 + term2, term1 + (surface(i, j + 1) + surface(i, j)) * thermal_cond / dr)

def b_j_temp_g(i, j):
    
    return - (surface(i, j+1) * thermal_cond / dr)

def c_j_temp_g(i, j):
    condition = (j == 0)
    
    return np.where(condition, 0, -surface(i, j) * thermal_cond / dr)

def B_j_g(t, i, j, n):
    condition = (j == 0)
    
    term1 = rho_cp_graph * volume(i, j) * T_graph[t, n, i, j] / dt + heat_coef_values[i] * surface(i, j) * T_salt[t,n, i] + frac(t)[i] * flux[n,i] / dz * power[t] * power_frac[n] * volume(i,j) / total_volume(i) * dz/channel_values[i] #* channel_values[0]/channel_values[i]
    term2 = rho_cp_graph * volume(i, j) * T_graph[t, n, i, j] / dt + frac(t)[i] * flux[n,i] / dz * power[t] * power_frac[n] * volume(i,j) / total_volume(i) * dz/channel_values[i] #* channel_values[0]/channel_values[i]
    
    return np.where(condition, term1, term2)

def heat_coef(n_elements):
    heat_coef = np.zeros(n_elements)
    heat_coef[:n_up] = 0.1
    heat_coef[(n_up+n_plenum):(n_up+n_plenum+n_down)] = 0.1
    return heat_coef

In [ ]:
# Pre-compute constant arrays
radius_values = radius(n_elements)
channel_values = channels(n_elements)
surface_salt_values = surface_salt(n_elements)
g_zt_values = g_zt(n_elements)
heat_ex_values = heat_ex(n_elements)
circumference = O(n_elements)
T_he_values = T_he(n_elements)
heat_coef_values = heat_coef(n_elements)

indices = np.arange(n_elements)
number_segments = 7

# Time parameters
t_start = 0
t_end = 1500 # End time of simulation in seconds
dt = 10  # Time step size in seconds
t_values = np.arange(t_start, t_end, dt)

# Creating correct size matrices
precursors = np.zeros((len(t_values), number_segments, n_elements, 6))
power = np.zeros(len(t_values)) 
T_salt = np.zeros((len(t_values), number_segments, n_elements))
T_graph = np.zeros((len(t_values), number_segments, n_elements, r_elements-1))
reactivity = np.zeros((len(t_values), number_segments))
beta_lost = np.zeros(len(t_values))

total_flux = flux_ratio + 6
power_frac = np.array([flux_ratio/total_flux, 1/total_flux, 1/total_flux, 1/total_flux, 1/total_flux, 1/total_flux, 1/total_flux])

# Initial conditions
power[0] = 250e6 # Initial power
for n in range(number_segments):
    for g in range(6):
        precursors[0,n,:,g] = beta_eff_groups[g] / (lambda_groups[g] * Lambda) * flux[n] / (dz * surface_salt_values) * power[0] * power_frac[n]  # Initial precursor concentrations for each group
T_salt[0] = T_0_s * np.ones(n_elements)  # Initial temperature of salt in Celsius
T_graph[0,:, :n_up, :] = T_0_g * np.ones((n_up, r_elements-1)) # Initial temperature of graphite in Celcius
T_graph[0,:, (n_up+n_plenum):(n_up + n_plenum + n_down), :] = T_0_g * np.ones((n_down, r_elements-1)) # Initial temperature of graphite in Celcius
reactivity[0,n] = alpha[n] * (np.sum(flux[n] * T_salt[0,n,:]) - T_ref[n])

# power[0] = power_end
# precursors[0] = precursor_end
# T_salt[0] = T_salt_end
# T_graph[0] = T_graph_end
# reactivity[0] = reactivity_end
# beta_lost[0] = beta_lost_end

# Initialize matrices for precursor and temperatures
A = np.zeros((n_elements, n_elements)) 
C = np.zeros((n_elements, n_elements))
E = np.zeros((n_elements, r_elements-1, r_elements-1))
F = np.zeros((r_elements-1))
sums = np.zeros(6)

# Time loop
start_time = time.time()


for t in range(len(t_values)-1):
    for n in range(number_segments):
        print(f'Iteration: {t}', end='\r')  # Print the current iteration number and overwrite it on the next iteration

        C[indices, indices] = a_j_temp_s(indices)
        C[indices, indices - 1] = b_j_temp_s(indices)
        D = B_j_s(t,n)

        T_salt[t + 1, n, :] = np.linalg.solve(C, D)

        for i in range(n_elements):
            if graphite_present(i):
                j = np.arange(r_elements-1)
                E[i, j, j] = a_j_temp_g(i, j)
                E[i, j[:-1], j[:-1] + 1] = b_j_temp_g(i, j[:-1])
                E[i, j, j - 1] = c_j_temp_g(i, j)
                F = B_j_g(t, i, j, n)
                T_graph[t + 1, n, i, :] = np.linalg.solve(E[i, :, :], F)
            else:
                T_graph[t + 1, n, i, :] = np.zeros(r_elements-1)

        reactivity[t+1, n] = alpha[n] * (np.sum(flux[n] * T_salt[t+1,n,:]) - T_ref[n]) 

        for g in range(6):
            A[indices, indices] = a_j_precursor(indices, g)
            A[indices, indices - 1] = b_j_precursor(indices)
            B = B_j_precursor(g, t,n)

            precursors[t+1,n,:,g] = np.linalg.solve(A, B)

    for g in range(6):
        sums[g] = lambda_groups[g] * np.sum(surface_salt_values * dz * np.sum(precursors[t+1,:,:,g], axis = 0) *1/np.sum(power_frac) * adjoint_flux_flat)# / np.sum(flux[n] * adjoint_flux_flat)
    beta_lost[t+1] = total_beta_eff - Lambda / (power[t]) * np.sum(sums)
    first_term = (1 / dt - (beta_lost[t+1] + np.sum(reactivity[t+1]) - total_beta_eff) / Lambda)**-1
    second_term = power[t] / dt + np.sum(sums)
    power[t+1] = first_term * second_term

end_time = time.time()
elapsed_time = end_time - start_time
print(f'The time it took to run {t} iterations was: {elapsed_time/60:.2f} minutes')

In [ ]:
Segment = 1
plt.plot(t_values[:], power[:])
# plt.title('Power over time')
plt.xlabel('Time [s]')
plt.ylabel('Power [W]')
# plt.grid()
# plt.savefig(r'Figures\Power over time stst', dpi = 300)
plt.show()

z_core = np.linspace(0, dz * (n_up + n_plenum + n_down), (n_up + n_plenum + n_down))
z_whole = np.linspace(0, dz * n_elements, n_elements)
graphtime = 10
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 0], label = 'Group 1')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 1], label = 'Group 2')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 2], label = 'Group 3')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 3], label = 'Group 4')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 4], label = 'Group 5')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 5], label = 'Group 6')
# plt.legend()
# plt.title(f'Precursor groups over core at t = {graphtime*dt}')
# plt.xlabel('z (cm)')
# plt.ylabel('Number of precursors')
# plt.grid()
# plt.show()

# plt.plot(z_whole, precursors[graphtime,Segment,:, 0], label = 'Group 1')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 1], label = 'Group 2')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 2], label = 'Group 3')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 3], label = 'Group 4')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 4], label = 'Group 5')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 5], label = 'Group 6')
# plt.legend()
# plt.title(f'Precursor groups over everything at t = {graphtime*dt}')
# plt.xlabel('z (cm)')
# plt.ylabel('Number of precursors')
# plt.grid()
# plt.show()

# plt.plot(t_values, precursors[:,Segment,0,0], label = 'Group 1')
# plt.plot(t_values, precursors[:,Segment,0,1], label = 'Group 2')
# plt.plot(t_values, precursors[:,Segment,0,2], label = 'Group 3')
# plt.plot(t_values, precursors[:,Segment,0,3], label = 'Group 4')
# plt.plot(t_values, precursors[:,Segment,0,4], label = 'Group 5')
# plt.plot(t_values, precursors[:,Segment,1,5], label = 'Group 6')
# plt.title('Precursor groups over time in first element')
# plt.xlabel('Time (s)')
# plt.ylabel('Number of precursors')
# plt.grid()
# plt.legend()
# plt.show()

plt.plot(t_values[:], reactivity[:, Segment])
# plt.title('Temperature reactivity over time')
plt.xlabel('Time [s]')
plt.ylabel('Reactivity')
# plt.grid()
# plt.savefig(r'Figures\Reactivity over time stst2', dpi = 300)
plt.show()

plt.plot(t_values[:], beta_lost[:])
plt.title('Eq reactivity over time')
plt.xlabel('Time (s)')
plt.ylabel('Reactivity')
plt.grid()
# plt.savefig(r'Figures\Reactivity over time', dpi = 300)
plt.show()

# plt.plot(t_values[:], beta_lost[:]+reactivity[:, Segment])
# plt.title('Total reactivity over time')
# plt.xlabel('Time (s)')
# plt.ylabel('Reactivity')
# plt.grid()
# # plt.savefig(r'Figures\Reactivity over time', dpi = 300)
# plt.show()

plt.plot(t_values[:], np.mean(T_salt[:,Segment], axis = 1))
# plt.title('Salt temperature over time')
plt.xlabel('Time [s]')
plt.ylabel('Temperature [K]')
# plt.grid()
# plt.savefig(r'Figures\Ts over time stst2', dpi = 300)
plt.show()

plt.plot(z_whole, T_salt[-1,Segment])
# plt.title('Salt temperature over reactor')
plt.xlabel('z (cm)')
plt.ylabel('Temperature [K]')
# plt.grid()
# plt.savefig(r'Figures\Ts over reactor stst2', dpi = 300)
plt.show()

plt.plot(t_values[:], np.mean(T_graph[:,Segment,:core_elements,1], axis = 1))
# plt.title('Graphite temperature over time')
plt.xlabel('Time [s]')
plt.ylabel('Temperature [K]')
# plt.grid()
# plt.savefig(r'Figures\Tg over time stst2', dpi = 300)
plt.show()

plt.plot(z_core, np.mean(T_graph[-1,Segment,:core_elements,:], axis = 1))
# plt.title('Graphite temperature over core')
plt.xlabel('z [cm]')
plt.ylabel('Temperature [K]')
# plt.grid()
# plt.savefig(r'Figures\Tg over core stst2', dpi = 300)
plt.show()

plt.plot(r(50)[:-1], T_graph[-1,Segment,50,:])
# plt.title('Graphite temperature as function of r')
plt.xlabel('Radius [cm]')
plt.ylabel('Temperature [K]')
# plt.grid()
# plt.savefig(r'Figures\Tg over r stst2', dpi = 300)
plt.show()

In [ ]:
Segment = 0

power_in_megawatts2 = (g_zt(t) * rho_cp_fuel * (np.max(T_salt[-1,Segment]) - np.min(T_salt[-1,Segment]))) * 1e-6
power_in_megawatts3 = power_frac[Segment]*power[-1]*1e-6

print(f'The power according to thermal hydraulics is: {power_in_megawatts2:.2f} MW')
print(f'The power according to the neutronics is: {power_in_megawatts3:.2f} MW')
print(f'The mean salt temperature at the end is {np.mean(T_salt[-1,Segment]):.3f} K')
a = T_graph[-1,Segment,:,0]
a = a[a != 0]
print(f'The mean graphite temperature at the end is {np.mean(a):.3f} K')

print(f'The time it takes to flow through the whole reactor is: {np.sum(surface_salt_values*dz)/g_zt(t):.2f} s')
print(f'The difference between the power from neutronics and thermal hydraulics is: {(power_in_megawatts3 - power_in_megawatts2)/power_in_megawatts2 *100} %')

# temp_diff_up = np.mean(frac(i)[0] * flux_flat[:n_up]/dz * power[-1] * 1 /(surface(10,j)[0] * heat_coef_values[:n_up]))
# temp_diff_down = np.mean(frac(i)[0] * flux_flat[(n_up+n_plenum):(n_up+n_plenum+n_down)]/dz * power[-1] * 1 /(surface(100,j)[0] * heat_coef_values[(n_up+n_plenum):(n_up+n_plenum+n_down)]))
# print(f'The temperature difference up should be: {temp_diff_up}')
# print(f'The temperature difference up is: {-np.mean(T_salt[-1,:n_up] - T_graph[-1,:n_up,0])}')
# print(f'The temperature difference down should be: {temp_diff_down}')
# print(f'The temperature difference down is: {-np.mean(T_salt[-1,(n_up+n_plenum):(n_up+n_plenum+n_down)] - T_graph[-1,(n_up+n_plenum):(n_up+n_plenum+n_down),0])}')

number = 100
print(f'Fraction of power is: {frac(number)[number] * flux[Segment,number]/dz * power[-1]*power_frac[Segment]}')
print(f'Heat flux is: {channel_values[number] / dz * heat_coef_values[number] * surface(number,j)[0] * (T_graph[-1,Segment,number,0] - T_salt[-1,Segment,number])}')

Power_up = np.sum(flux[Segment,:n_up]) * power[-1]*1e-6 *power_frac[Segment]
Power_down = np.sum(flux[Segment,n_up+n_plenum:n_up+n_plenum+n_down]) * power[-1]*1e-6*power_frac[Segment]

T_up = g_zt_values*rho_cp_fuel*(T_salt[-1,Segment,n_up-1]-T_salt[-1,Segment,-1])*1e-6
T_down = g_zt_values*rho_cp_fuel*(T_salt[-1,Segment,n_up+n_plenum+n_down+1]-T_salt[-1,Segment,n_up+n_plenum-1])*1e-6
print(f'The power up is: {Power_up:.2f}')
print(f'The temperature increase power up is: {T_up:.2f}')

print(f'The power down is: {Power_down:.2f}')
print(f'The temperature increase power down is: {T_down:.2f}')

print(f'The total power is: {power[-1]*1e-6:.2f} MW')
print(f'The power in the middle segment is {power_frac[0]*power[-1]*1e-6:.2f} MW and in the outer segment is {power_frac[1]*power[-1]*1e-6:.2f} MW')

print(np.sum(flux[Segment] * T_salt[t+1,Segment,:]))
print(np.mean(T_salt[-1, Segment, :core_elements]))

In [ ]:
power_end = power[-1]
precursor_end = precursors[-1]
T_salt_end = T_salt[-1]
T_graph_end = T_graph[-1]
reactivity_end = reactivity[-1]
beta_lost_end = beta_lost[-1]

In [ ]:
def frac(t):
    frac = np.zeros(n_elements)
    frac[:n_up] = 0.05
    frac[(n_up+n_plenum):(n_up + n_plenum + n_down)] = 0.05
    return frac

def g_zt(t):
    return 100000  #80247 # cm^3/s

def radius(n_elements): # Radius difference for up and down channels
    radius = np.ones(n_elements)
    radius[:n_up] = 8
    radius[(n_up + n_plenum):(n_up + n_plenum + n_down)] = 0.5 * 8
    return radius

def graphite_present(i):
    if 0 <= i < n_up or (n_up + n_plenum) <= i < (n_up + n_plenum + n_down):
        return True
    else:
        return False

# Cross-sectional area function
def channels(n_elements):
    channels = np.zeros(n_elements)
    channels[:n_up] = 6
    channels[n_up:(n_up + n_plenum)] = 1
    channels[(n_up+n_plenum):(n_up + n_plenum + n_down)] = 24
    channels[(n_up + n_plenum + n_down):] = 1
    return channels
    
def surface_salt(i):
    surface = np.zeros(n_elements)
    surface[:n_up] = channel_values[1] *radius_values[1]**2*np.pi
    surface[n_up:(n_up + n_plenum)] = 100*np.pi
    surface[(n_up+n_plenum):(n_up + n_plenum + n_down)] = channel_values[n_up+n_plenum+2] * radius_values[n_up+n_plenum+2]**2 * np.pi
#     surface[(n_up + n_plenum + n_down):] = 150*np.pi
    surface[(n_up + n_plenum + n_down):] = OL/L
    return surface

def volume(i, j):
    r_ = r(i) - dr/2
    volume = np.array([np.pi * dz * (r_[idx+1]**2 - r_[idx]**2) for idx in j])
    return volume

def total_volume(i):
    return np.pi * ((r(i)[-1]-dr/2)**2 - (r(i)[0]-dr/2)**2) * dz

def surface(i, j):
    r_ = r(i) - dr/2
    surface = np.array([0 if idx == (r_elements-1) else 2*np.pi * r_[idx] * dz for idx in j])
    return surface

def total_surface(i):
    return total_volume(i)/dz
    
def r(i):
    if i <= n_up:
        return np.linspace(radius_values[0], radius_values[0] + e1, r_elements) + dr/2
    if (n_up + n_plenum) <= i <= (n_up + n_plenum + n_down):
        return np.linspace(radius_values[(n_up + n_plenum + 1)], radius_values[(n_up + n_plenum + 1)] + e2, r_elements) + dr/2
    else:
        return np.zeros(r_elements)

# Heat exchanger functions
def heat_ex(n_elements):
    heat_ex = np.zeros((number_segments, n_elements))
#     heat_ex[int(-(LL/2+L)/dz):int(-(LL/2)/dz)] = 0.3486
    heat_ex[:, int(-(LL/2+L)/dz):int(-(LL/2)/dz)] = 5
    heat_ex[0,int(-(LL/2+L)/dz):int(-(LL/2)/dz)] = 10
    return heat_ex

def O(n_elements):
    O = np.zeros(n_elements)
    O[int(-(LL/2+L)/dz):int(-(LL/2)/dz)] = OL / L
    return O

def T_he(n_elements):
    T_he = np.zeros((len(t_values), number_segments, n_elements))
    T_he[:, :, int(-(LL/2+L)/dz):int(-(LL/2)/dz)] = 700 # np.linspace(466.7, 436.1, int(L/dz)) + 273.15
    return T_he

# Matrix elements
def a_j_precursor(i, group,n):
    return 1 / dt + lambda_groups[group] + g_zt_values[n] / (surface_salt_values[i] * dz)

def b_j_precursor(i,n):
    return - g_zt_values[n] / (surface_salt_values[i] * dz)

def B_j_precursor(group, t, n):
    return beta_eff_groups[group] / Lambda * flux[n] / (surface_salt_values * dz) * power[t] * power_frac[n] + precursors[t,n, :, group] / dt

def a_j_temp_s(i,n):
    return surface_salt_values[i] / dt + g_zt_values[n] / dz + heat_ex_values[n,i] * circumference[i] / rho_cp_fuel + channel_values[i] * 2*np.pi*radius_values[i]* heat_coef_values[i] / (rho_cp_fuel)

def b_j_temp_s(i,n):
    return - g_zt_values[n] / dz

def B_j_s(t, n):
    return (1 - frac(t)) * 1 / (rho_cp_fuel) * flux[n] / dz * power[t] * power_frac[n] + T_he_values[t,n] * heat_ex_values[n] * circumference / (rho_cp_fuel) + channel_values * 2*np.pi*radius_values* heat_coef_values * T_graph[t,n,:,0] / (rho_cp_fuel) + surface_salt_values / dt * T_salt[t,n,:]

def a_j_temp_g(i, j):
    condition = (j == 0)
    
    term1 = rho_cp_graph * (volume(i, j)) / dt
    term2 = (surface(i, j + 1) * thermal_cond / dr + heat_coef_values[i] * surface(i, j))

    return np.where(condition, term1 + term2, term1 + (surface(i, j + 1) + surface(i, j)) * thermal_cond / dr)

def b_j_temp_g(i, j):
    
    return - (surface(i, j+1) * thermal_cond / dr)

def c_j_temp_g(i, j):
    condition = (j == 0)
    
    return np.where(condition, 0, -surface(i, j) * thermal_cond / dr)

def B_j_g(t, i, j, n):
    condition = (j == 0)
    
    term1 = rho_cp_graph * volume(i, j) * T_graph[t, n, i, j] / dt + heat_coef_values[i] * surface(i, j) * T_salt[t,n, i] + frac(t)[i] * flux[n,i] / dz * power[t] * power_frac[n] * volume(i,j) / total_volume(i) * dz/channel_values[i] #* channel_values[0]/channel_values[i]
    term2 = rho_cp_graph * volume(i, j) * T_graph[t, n, i, j] / dt + frac(t)[i] * flux[n,i] / dz * power[t] * power_frac[n] * volume(i,j) / total_volume(i) * dz/channel_values[i] #* channel_values[0]/channel_values[i]
    
    return np.where(condition, term1, term2)

# Functions for heat exchanger coefficient
# def f_Pr(Pr):
#     return 0.886 / (1 + (1.909 * Pr**(1/6))**9/2)**2/9

# def fRe(eps):
#     return 12 / (np.sqrt(eps) * (1 + eps) * (1 - 192 * eps / np.pi**5 * np.tanh(np.pi / (2 * eps))))
    
# def Nu(z, Pr, eps, m):
#     return ((f_Pr(Pr) / np.sqrt(z)) + ((0.501 * (fRe(eps) / z)**(1/3))**5 + (3.86 * fRe(eps) / (8 * np.sqrt(np.pi)))**5)**(m/5))**(1/m)
    
def heat_coef(n_elements):
    heat_coef = np.zeros(n_elements)
    heat_coef[:n_up] = 0.5
    heat_coef[(n_up+n_plenum):(n_up+n_plenum+n_down)] = 0.5
    return heat_coef

In [ ]:
number_segments = 7

# Time parameters
t_start = 0
t_end = 500 # End time of simulation in seconds
dt = 1  # Time step size in seconds
t_values = np.arange(t_start, t_end, dt)

# Pre-compute constant arrays
radius_values = radius(n_elements)
channel_values = channels(n_elements)
surface_salt_values = surface_salt(n_elements)
g_zt_values = g_zt(n_elements)*np.ones(number_segments)
heat_ex_values = heat_ex(n_elements)
circumference = O(n_elements)
T_he_values = T_he(n_elements)
heat_coef_values = heat_coef(n_elements)
A_he = np.pi * (10.5**2-7.5**2)

indices = np.arange(n_elements)

# Creating correct size matrices
precursors = np.zeros((len(t_values), number_segments, n_elements, 6))
power = np.zeros(len(t_values)) 
T_salt = np.zeros((len(t_values), number_segments, n_elements))
T_graph = np.zeros((len(t_values), number_segments, n_elements, r_elements-1))
reactivity = np.zeros((len(t_values), number_segments))
beta_lost = np.zeros(len(t_values))

flux_dict = {}
power_frac_dict = {}
flux_t_list = np.array([0,50,100])#,200,400,600,800])

for i in flux_t_list:
    # Load flux data for the current time value
    flux1 = np.load(rf'FSOC\FSOCfission1_{i}.npy')
    flux2 = np.load(rf'FSOC\FSOCfission2_{i}.npy')
#     flux3 = np.load(rf'FSOC3\FSOC3fission3_{i}.npy')
#     flux4 = np.load(rf'FSOC3\FSOC3fission4_{i}.npy')
#     flux5 = np.load(rf'FSOC3\FSOC3fission5_{i}.npy')
#     flux6 = np.load(rf'FSOC3\FSOC3fission6_{i}.npy')
#     flux7 = np.load(rf'FSOC3\FSOC3fission7_{i}.npy')

    # Invert flux2
    flux_inverted1 = flux1[::-1]
    flux_inverted2 = flux2[::-1]
#     flux_inverted3 = flux3[::-1]
#     flux_inverted4 = flux4[::-1]
#     flux_inverted5 = flux5[::-1]
#     flux_inverted6 = flux6[::-1]
#     flux_inverted7 = flux7[::-1]

    # Concatenate flux1, inverted flux2, and zeros
    flux1 = np.concatenate([flux1, flux_inverted, np.zeros(int(L_he/dz))])
    flux2 = np.concatenate([flux2, flux_inverted, np.zeros(int(L_he/dz))])
#     flux3 = np.concatenate([flux3, flux_inverted, np.zeros(int(L_he/dz))])
#     flux4 = np.concatenate([flux4, flux_inverted, np.zeros(int(L_he/dz))])
#     flux5 = np.concatenate([flux5, flux_inverted, np.zeros(int(L_he/dz))])
#     flux6 = np.concatenate([flux6, flux_inverted, np.zeros(int(L_he/dz))])
#     flux7 = np.concatenate([flux7, flux_inverted, np.zeros(int(L_he/dz))])


    total_flux1 = np.sum(flux1)
    total_flux2 = np.sum(flux2)
#     total_flux3 = np.sum(flux3)
#     total_flux4 = np.sum(flux4)
#     total_flux5 = np.sum(flux5)
#     total_flux6 = np.sum(flux6)
#     total_flux7 = np.sum(flux7)
    total_flux = total_flux1 + total_flux2 * 6
#     total_flux = total_flux1 + total_flux2 + total_flux3 + total_flux4 + total_flux5 + total_flux6 + total_flux7

    # Calculate the power fraction
    power_frac1 = total_flux1 / total_flux
    power_frac2 = total_flux2 / total_flux
#     power_frac3 = total_flux3 / total_flux
#     power_frac4 = total_flux4 / total_flux
#     power_frac5 = total_flux5 / total_flux
#     power_frac6 = total_flux6 / total_flux
#     power_frac7 = total_flux7 / total_flux
    
    flux1 /= total_flux1
    flux2 /= total_flux2
#     flux3 /= total_flux3
#     flux4 /= total_flux4
#     flux5 /= total_flux5
#     flux6 /= total_flux6
#     flux7 /= total_flux7

    # Assign other fluxes with the same values as flux2
    flux3 = flux2
    flux4 = flux2
    flux5 = flux2
    flux6 = flux2
    flux7 = flux2

    # Stack the fluxes
    flux = np.vstack((flux1, flux2, flux3, flux4, flux5, flux6, flux7))

    # Store flux data for the current time value
    flux_dict[i] = flux
    power_frac_dict[i] = np.array([power_frac1, power_frac2, power_frac2, power_frac2, power_frac2, power_frac2, power_frac2])
#     power_frac_dict[i] = np.array([power_frac1, power_frac2, power_frac3, power_frac4, power_frac5, power_frac6, power_frac7])

    
power[0] = power_end
precursors[0] = precursor_end
T_salt[0] = T_salt_end
T_graph[0] = T_graph_end
reactivity[0] = reactivity_end
beta_lost[0] = beta_lost_end

# Initialize matrices for precursor and temperatures
A = np.zeros((n_elements, n_elements)) 
C = np.zeros((n_elements, n_elements))
E = np.zeros((n_elements, r_elements-1, r_elements-1))
F = np.zeros((r_elements-1))
sums = np.zeros(6)

# Time loop
start_time = time.time()

for t in range(len(t_values)-1):
    if t in flux_t_list:
        flux = flux_dict[t]
        power_frac = power_frac_dict[t]
    for n in range(number_segments):
        print(f'Iteration: {t}', end='\r')  # Print the current iteration number and overwrite it on the next iteration
        
        C[indices, indices] = a_j_temp_s(indices,n)
        C[indices, indices - 1] = b_j_temp_s(indices,n)
        D = B_j_s(t,n)

        T_salt[t + 1, n, :] = np.linalg.solve(C, D)

        for i in range(n_elements):
            if graphite_present(i):
                j = np.arange(r_elements-1)
                E[i, j, j] = a_j_temp_g(i, j)
                E[i, j[:-1], j[:-1] + 1] = b_j_temp_g(i, j[:-1])
                E[i, j, j - 1] = c_j_temp_g(i, j)
                F = B_j_g(t, i, j, n)
                T_graph[t + 1, n, i, :] = np.linalg.solve(E[i, :, :], F)
            else:
                T_graph[t + 1, n, i, :] = np.zeros(r_elements-1)

        reactivity[t+1, n] = alpha[n] * (np.sum(flux[n] * T_salt[t+1,n,:]) - T_ref[n]) 
        
        for g in range(6):
            A[indices, indices] = a_j_precursor(indices, g,n)
            A[indices, indices - 1] = b_j_precursor(indices,n)
            B = B_j_precursor(g, t,n)

            precursors[t+1,n,:,g] = np.linalg.solve(A, B)
            
#     T_he_values[t+1,1] = ((A_he * rho_cp_he)/dt + heat_ex_values*circumference)**-1 * (heat_ex_values * circumference * T_salt[t+1,1,:] + (A_he*rho_cp_he)/dt * T_he_values[t,1])

    for g in range(6):
        sums[g] = lambda_groups[g] * np.sum(surface_salt_values * dz * np.sum(precursors[t+1,:,:,g], axis = 0) *1/np.sum(power_frac) * adjoint_flux_flat)# / np.sum(flux[n] * adjoint_flux_flat)
    beta_lost[t+1] = total_beta_eff - Lambda / (power[t]) * np.sum(sums)
    first_term = (1 / dt - (beta_lost[t+1] + np.sum(reactivity[t+1]) - total_beta_eff) / Lambda)**-1
    second_term = power[t] / dt + np.sum(sums)
    power[t+1] = first_term * second_term
    
end_time = time.time()
elapsed_time = end_time - start_time
print(f'The time it took to run {t} iterations was: {elapsed_time/60:.2f} minutes')

In [ ]:
number = -1
Segment = 0
plt.plot(t_values[:number], power[:number])
# plt.title('Power over time')
plt.xlabel('Time [s]')
plt.ylabel('Power')
# plt.grid()
# plt.savefig(r'Figures\Power over time 3FSOC', dpi = 300)
plt.show()

z_core = np.linspace(0, dz * (n_up + n_plenum + n_down), (n_up + n_plenum + n_down))
z_whole = np.linspace(0, dz * n_elements, n_elements)
graphtime = 10
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 0], label = 'Group 1')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 1], label = 'Group 2')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 2], label = 'Group 3')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 3], label = 'Group 4')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 4], label = 'Group 5')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 5], label = 'Group 6')
# plt.legend()
# plt.title(f'Precursor groups over core at t = {graphtime*dt}')
# plt.xlabel('z (cm)')
# plt.ylabel('Number of precursors')
# plt.grid()
# plt.show()

# plt.plot(z_whole, precursors[graphtime,Segment,:, 0], label = 'Group 1')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 1], label = 'Group 2')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 2], label = 'Group 3')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 3], label = 'Group 4')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 4], label = 'Group 5')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 5], label = 'Group 6')
# plt.legend()
# plt.title(f'Precursor groups over everything at t = {graphtime*dt}')
# plt.xlabel('z (cm)')
# plt.ylabel('Number of precursors')
# plt.grid()
# plt.show()

# plt.plot(t_values, precursors[:,Segment,0,0], label = 'Group 1')
# plt.plot(t_values, precursors[:,Segment,0,1], label = 'Group 2')
# plt.plot(t_values, precursors[:,Segment,0,2], label = 'Group 3')
# plt.plot(t_values, precursors[:,Segment,0,3], label = 'Group 4')
# plt.plot(t_values, precursors[:,Segment,0,4], label = 'Group 5')
# plt.plot(t_values, precursors[:,Segment,1,5], label = 'Group 6')
# plt.title('Precursor groups over time in first element')
# plt.xlabel('Time (s)')
# plt.ylabel('Number of precursors')
# plt.grid()
# plt.legend()
# plt.show()

plt.plot(t_values[:number], reactivity[:number, Segment])
# plt.title('Temperature reactivity over time')
plt.xlabel('Time [s]')
plt.ylabel('Reactivity')
# plt.grid()
plt.tight_layout()
# plt.savefig(r'Figures\Reactivity over time 2FSOC1', bbox_inches='tight', dpi = 300)
plt.show()

# plt.plot(t_values[:200], beta_lost[:200])
# plt.title('Eq reactivity over time')
# plt.xlabel('Time (s)')
# plt.ylabel('Reactivity')
# plt.grid()
# plt.tight_layout()
# # plt.savefig(r'Figures\Reactivity over time', bbox_inches='tight', dpi = 300)
# plt.show()

# plt.plot(t_values[:200], beta_lost[:200]+reactivity[:200])
# plt.title('Total reactivity over time')
# plt.xlabel('Time (s)')
# plt.ylabel('Reactivity')
# plt.grid()
# plt.tight_layout()
# # plt.savefig(r'Figures\Reactivity over time', bbox_inches='tight', dpi = 300)
# plt.show()

plt.plot(t_values[:number], np.mean(T_salt[:number,Segment], axis = 1))
# plt.title('Salt temperature over time')
plt.xlabel('Time [s]')
plt.ylabel('Temperature [K]')
# plt.grid()
# plt.savefig(r'Figures\Ts over time 3FSOC3', dpi = 300)
plt.show()

plt.plot(z_whole, T_salt[-1,Segment])
# plt.title('Salt temperature over reactor')
plt.xlabel('z [cm])')
plt.ylabel('Temperature [K]')
# plt.grid()
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.tight_layout()
# plt.savefig(r'Figures\Ts over reactor 3FSOC3', bbox_inches='tight', dpi = 300)
plt.show()

plt.plot(t_values[:number], np.mean(T_graph[:number,Segment,n_up+n_plenum:core_elements,1], axis = 1))
# plt.title('Graphite temperature over time')
plt.xlabel('Time [s]')
plt.ylabel('Temperature [K]')
# plt.grid()
# plt.savefig(r'Figures\Tg over time 3FSOC3', dpi = 300)
plt.show()

plt.plot(z_core, np.mean(T_graph[-1,Segment,:core_elements,:], axis = 1))
# plt.title('Graphite temperature over core')
plt.xlabel('z [cm]')
plt.ylabel('Temperature [K]')
# plt.grid()
# plt.savefig(r'Figures\Tg over core 3FSOC3', dpi = 300)
plt.show()

plt.plot(r(50)[:-1], T_graph[-1,Segment,50,:])
# plt.title('Graphite temperature as function of r')
plt.xlabel('Radius [cm]')
plt.ylabel('Temperature [K]')
# plt.grid()
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
# plt.savefig(r'Figures\Tg over r 3FSOC3', dpi = 300)
plt.show()

In [ ]:
Segment = 0

power_in_megawatts2 = (g_zt(t) * rho_cp_fuel * (np.max(T_salt[-1,Segment]) - np.min(T_salt[-1,Segment]))) * 1e-6
power_in_megawatts3 = power_frac[Segment]*power[-1]*1e-6

print(f'The power according to thermal hydraulics is: {power_in_megawatts2:.2f} MW')
print(f'The power according to the neutronics is: {power_in_megawatts3:.2f} MW')
print(f'The mean salt temperature at the end is {np.mean(T_salt[-1,Segment]):.3f} K')
a = T_graph[-1,Segment,:,0]
a = a[a != 0]
print(f'The mean graphite temperature at the end is {np.mean(a):.3f} K')

print(f'The time it takes to flow through the whole reactor is: {np.sum(surface_salt_values*dz)/g_zt(t):.2f} s')
print(f'The difference between the power from neutronics and thermal hydraulics is: {(power_in_megawatts3 - power_in_megawatts2)/power_in_megawatts2 *100} %')

# temp_diff_up = np.mean(frac(i)[0] * flux_flat[:n_up]/dz * power[-1] * 1 /(surface(10,j)[0] * heat_coef_values[:n_up]))
# temp_diff_down = np.mean(frac(i)[0] * flux_flat[(n_up+n_plenum):(n_up+n_plenum+n_down)]/dz * power[-1] * 1 /(surface(100,j)[0] * heat_coef_values[(n_up+n_plenum):(n_up+n_plenum+n_down)]))
# print(f'The temperature difference up should be: {temp_diff_up}')
# print(f'The temperature difference up is: {-np.mean(T_salt[-1,:n_up] - T_graph[-1,:n_up,0])}')
# print(f'The temperature difference down should be: {temp_diff_down}')
# print(f'The temperature difference down is: {-np.mean(T_salt[-1,(n_up+n_plenum):(n_up+n_plenum+n_down)] - T_graph[-1,(n_up+n_plenum):(n_up+n_plenum+n_down),0])}')

number = 100
print(f'Fraction of power is: {frac(number)[number] * flux[Segment,number]/dz * power[-1]*power_frac[Segment]}')
print(f'Heat flux is: {channel_values[number] / dz * heat_coef_values[number] * surface(number,j)[0] * (T_graph[-1,Segment,number,0] - T_salt[-1,Segment,number])}')

Power_up = np.sum(flux[Segment,:n_up]) * power[-1]*1e-6 *power_frac[Segment]
Power_down = np.sum(flux[Segment,n_up+n_plenum:n_up+n_plenum+n_down]) * power[-1]*1e-6*power_frac[Segment]

T_up = g_zt_values*rho_cp_fuel*(T_salt[-1,Segment,n_up-1]-T_salt[-1,Segment,-1])*1e-6
T_down = g_zt_values*rho_cp_fuel*(T_salt[-1,Segment,n_up+n_plenum+n_down+1]-T_salt[-1,Segment,n_up+n_plenum-1])*1e-6
# print(f'The power up is: {Power_up:.2f}')
# print(f'The temperature increase power up is: {T_up:.2f}')

# print(f'The power down is: {Power_down:.2f}')
# print(f'The temperature increase power down is: {T_down:.2f}')

print(f'The total power is: {power[-1]*1e-6:.2f} MW')
print(f'The power in the middle segment is {power_frac[0]*power[-1]*1e-6:.2f} MW and in the outer segment is {power_frac[1]*power[-1]*1e-6:.2f} MW')

In [ ]:
print(np.mean(T_salt[50,0,:core_elements]))
print(np.mean(T_salt[50,1,:core_elements]))
# print(np.mean(T_salt[400,2,:core_elements]))
# print(np.mean(T_salt[50,3,:core_elements]))
# print(np.mean(T_salt[50,4,:core_elements]))
# print(np.mean(T_salt[50,5,:core_elements]))
# print(np.mean(T_salt[50,6,:core_elements]))

a = T_graph[50,0,:,:]
a = a[a != 0]
print(f'The mean graphite temperature at the end is {np.mean(a):.3f} K')

b = T_graph[50,1,:,:]
b = b[b != 0]
print(f'The mean graphite temperature at the end is {np.mean(b):.3f} K')

# b = T_graph[400,2,:,:]
# b = b[b != 0]
# print(f'The mean graphite temperature at the end is {np.mean(b):.3f} K')

# b = T_graph[50,3,:,:]
# b = b[b != 0]
# print(f'The mean graphite temperature at the end is {np.mean(b):.3f} K')

# b = T_graph[50,4,:,:]
# b = b[b != 0]
# print(f'The mean graphite temperature at the end is {np.mean(b):.3f} K')

# b = T_graph[50,5,:,:]
# b = b[b != 0]
# print(f'The mean graphite temperature at the end is {np.mean(b):.3f} K')

# b = T_graph[50,6,:,:]
# b = b[b != 0]
# print(f'The mean graphite temperature at the end is {np.mean(b):.3f} K')

In [ ]:
def frac(t):
    frac = np.zeros(n_elements)
    frac[:n_up] = 0.05
    frac[(n_up+n_plenum):(n_up + n_plenum + n_down)] = 0.05
    return frac

def g_zt(t):
    g_zt = 100000*np.ones((len(t_values), number_segments))
    a = np.linspace(100000, 20000, 80)
#     g_zt[:80,1:] = np.tile(a[:, None], (1, number_segments-1))
    g_zt[:80,0] = a
    g_zt[80:,0] = a[-1]
    return g_zt 

def radius(n_elements): # Radius difference for up and down channels
    radius = np.ones(n_elements)
    radius[:n_up] = 8
    radius[(n_up + n_plenum):(n_up + n_plenum + n_down)] = 0.5 * 8
    return radius

def graphite_present(i):
    if 0 <= i < n_up or (n_up + n_plenum) <= i < (n_up + n_plenum + n_down):
        return True
    else:
        return False

# Cross-sectional area function
def channels(n_elements):
    channels = np.zeros(n_elements)
    channels[:n_up] = 6
    channels[n_up:(n_up + n_plenum)] = 1
    channels[(n_up+n_plenum):(n_up + n_plenum + n_down)] = 24
    channels[(n_up + n_plenum + n_down):] = 1
    return channels
    
def surface_salt(i):
    surface = np.zeros(n_elements)
    surface[:n_up] = channel_values[1] *radius_values[1]**2*np.pi
    surface[n_up:(n_up + n_plenum)] = 100*np.pi
    surface[(n_up+n_plenum):(n_up + n_plenum + n_down)] = channel_values[n_up+n_plenum+2] * radius_values[n_up+n_plenum+2]**2 * np.pi
    surface[(n_up + n_plenum + n_down):] = OL/L
#     surface[(n_up + n_plenum + n_down):] = 150*np.pi
    return surface

def volume(i, j):
    r_ = r(i) - dr/2
    volume = np.array([np.pi * dz * (r_[idx+1]**2 - r_[idx]**2) for idx in j])
    return volume

def total_volume(i):
    return np.pi * ((r(i)[-1]-dr/2)**2 - (r(i)[0]-dr/2)**2) * dz

def surface(i, j):
    r_ = r(i) - dr/2
    surface = np.array([0 if idx == (r_elements-1) else 2*np.pi * r_[idx] * dz for idx in j])
    return surface

def total_surface(i):
    return total_volume(i)/dz
    
def r(i):
    if i <= n_up:
        return np.linspace(radius_values[0], radius_values[0] + e1, r_elements) + dr/2
    if (n_up + n_plenum) <= i <= (n_up + n_plenum + n_down):
        return np.linspace(radius_values[(n_up + n_plenum + 1)], radius_values[(n_up + n_plenum + 1)] + e2, r_elements) + dr/2
    else:
        return np.zeros(r_elements)

# Heat exchanger functions
def heat_ex(n_elements):
    heat_ex = np.zeros(n_elements)
#     heat_ex[int(-(LL/2+L)/dz):int(-(LL/2)/dz)] = 0.3486
    heat_ex[int(-(LL/2+L)/dz):int(-(LL/2)/dz)] = 5
    return heat_ex

def O(n_elements):
    O = np.zeros(n_elements)
    O[int(-(LL/2+L)/dz):int(-(LL/2)/dz)] = OL / L
    return O

def T_he(n_elements):
    T_he = np.zeros(n_elements)
    T_he[int(-(LL/2+L)/dz):int(-(LL/2)/dz)] = 700 # np.linspace(466.7, 436.1, int(L/dz)) + 273.15
    return T_he

# Matrix elements
def a_j_precursor(i, group, t, n):
    return 1 / dt + lambda_groups[group] + g_zt_values[t,n] / (surface_salt_values[i] * dz)

def b_j_precursor(i, t, n):
    return - g_zt_values[t,n] / (surface_salt_values[i] * dz)

def B_j_precursor(group, t, n):
    return beta_eff_groups[group] / Lambda * flux[n] / (surface_salt_values * dz) * power[t] * power_frac[n] + precursors[t,n, :, group] / dt

def a_j_temp_s(i, t, n):
    return surface_salt_values[i] / dt + g_zt_values[t,n] / dz + heat_ex_values[i] * circumference[i] / rho_cp_fuel + channel_values[i] * 2*np.pi*radius_values[i]* heat_coef_values[i] / (rho_cp_fuel)

def b_j_temp_s(i, t, n):
    return - g_zt_values[t,n] / dz

def B_j_s(t, n):
    return (1 - frac(t)) * 1 / (rho_cp_fuel) * flux[n] / dz * power[t] * power_frac[n] + T_he_values * heat_ex_values * circumference / (rho_cp_fuel) + channel_values * 2*np.pi*radius_values* heat_coef_values * T_graph[t,n,:,0] / (rho_cp_fuel) + surface_salt_values / dt * T_salt[t,n,:]

def a_j_temp_g(i, j):
    condition = (j == 0)
    
    term1 = rho_cp_graph * (volume(i, j)) / dt
    term2 = (surface(i, j + 1) * thermal_cond / dr + heat_coef_values[i] * surface(i, j))

    return np.where(condition, term1 + term2, term1 + (surface(i, j + 1) + surface(i, j)) * thermal_cond / dr)

def b_j_temp_g(i, j):
    
    return - (surface(i, j+1) * thermal_cond / dr)

def c_j_temp_g(i, j):
    condition = (j == 0)
    
    return np.where(condition, 0, -surface(i, j) * thermal_cond / dr)

def B_j_g(t, i, j, n):
    condition = (j == 0)
    
    term1 = rho_cp_graph * volume(i, j) * T_graph[t, n, i, j] / dt + heat_coef_values[i] * surface(i, j) * T_salt[t,n, i] + frac(t)[i] * flux[n,i] / dz * power[t] * power_frac[n] * volume(i,j) / total_volume(i) * dz/channel_values[i] #* channel_values[0]/channel_values[i]
    term2 = rho_cp_graph * volume(i, j) * T_graph[t, n, i, j] / dt + frac(t)[i] * flux[n,i] / dz * power[t] * power_frac[n] * volume(i,j) / total_volume(i) * dz/channel_values[i] #* channel_values[0]/channel_values[i]
    
    return np.where(condition, term1, term2)

def heat_coef(n_elements):
    heat_coef = np.zeros(n_elements)
    heat_coef[:n_up] = 0.1
    heat_coef[(n_up+n_plenum):(n_up+n_plenum+n_down)] = 0.1
    return heat_coef

In [ ]:
time1 = np.array([0.0, 0.45, 0.75, 1.5, 2.25, 3.0, 3.75, 4.5, 5.25, 6.0])#,, 10.5, 15.0])
g_nom = 100000  # Constant salt volume flow rate in cm^3/s
fuel_flow_rate = np.array([100, 98, 95, 80, 57, 42, 28, 18, 13, 10])* g_nom / 100 #, 2.5, 0]) 

dt = 0.1

extended_time = np.arange(0.0, 70.0 + dt, dt)

interpolated_flow_rate = np.interp(extended_time, time1, fuel_flow_rate)

new_fuel_flow_rate = np.copy(interpolated_flow_rate)
new_fuel_flow_rate[extended_time > 15] = 0.10*g_nom  # Setting values after 15 seconds to zero

# Plotting the fuel flow rate vs. time
plt.plot(extended_time, new_fuel_flow_rate)
plt.xlabel("Time [s]")
plt.ylabel(r"Fuel Flow Rate [% of $g_{nom}$]")
plt.title("Fuel Flow Rate vs. Time")
# plt.grid(True)
plt.show()

In [ ]:
# Time parameters
t_start = 0
t_end = 250 # End time of simulation in seconds
dt = 1  # Time step size in seconds
t_values = np.arange(t_start, t_end, dt)

indices = np.arange(n_elements)
number_segments = 7

# Pre-compute constant arrays
radius_values = radius(n_elements)
channel_values = channels(n_elements)
surface_salt_values = surface_salt(n_elements)
g_zt_values = g_zt(t)
heat_ex_values = heat_ex(n_elements)
circumference = O(n_elements)
T_he_values = T_he(n_elements)
heat_coef_values = heat_coef(n_elements)

# Creating correct size matrices
precursors = np.zeros((len(t_values), number_segments, n_elements, 6))
power = np.zeros(len(t_values)) 
T_salt = np.zeros((len(t_values), number_segments, n_elements))
T_graph = np.zeros((len(t_values), number_segments, n_elements, r_elements-1))
reactivity = np.zeros((len(t_values), number_segments))
beta_lost = np.zeros(len(t_values))

flux_dict = {}
power_frac_dict = {}
flux_t_list = np.array([0,50,100])#,100,200,400,600,800])

for i in flux_t_list:
    # Load flux data for the current time value
    flux1 = np.load(rf'flow1\flow1fission1_{i}.npy')
    flux2 = np.load(rf'flow1\flow1fission2_{i}.npy')
#     flux3 = np.load(rf'ULOHS3\ULOHS3fission3_{i}.npy')
#     flux4 = np.load(rf'ULOHS3\ULOHS3fission4_{i}.npy')
#     flux5 = np.load(rf'ULOHS3\ULOHS3fission5_{i}.npy')
#     flux6 = np.load(rf'ULOHS3\ULOHS3fission6_{i}.npy')
#     flux7 = np.load(rf'ULOHS3\ULOHS3fission7_{i}.npy')

    # Invert flux2
    flux_inverted1 = flux1[::-1]
    flux_inverted2 = flux2[::-1]
#     flux_inverted3 = flux3[::-1]
#     flux_inverted4 = flux4[::-1]
#     flux_inverted5 = flux5[::-1]
#     flux_inverted6 = flux6[::-1]
#     flux_inverted7 = flux7[::-1]

    # Concatenate flux1, inverted flux2, and zeros
    flux1 = np.concatenate([flux1, flux_inverted, np.zeros(int(L_he/dz))])
    flux2 = np.concatenate([flux2, flux_inverted, np.zeros(int(L_he/dz))])
#     flux3 = np.concatenate([flux3, flux_inverted, np.zeros(int(L_he/dz))])
#     flux4 = np.concatenate([flux4, flux_inverted, np.zeros(int(L_he/dz))])
#     flux5 = np.concatenate([flux5, flux_inverted, np.zeros(int(L_he/dz))])
#     flux6 = np.concatenate([flux6, flux_inverted, np.zeros(int(L_he/dz))])
#     flux7 = np.concatenate([flux7, flux_inverted, np.zeros(int(L_he/dz))])


    total_flux1 = np.sum(flux1)
    total_flux2 = np.sum(flux2)
#     total_flux3 = np.sum(flux3)
#     total_flux4 = np.sum(flux4)
#     total_flux5 = np.sum(flux5)
#     total_flux6 = np.sum(flux6)
#     total_flux7 = np.sum(flux7)
    total_flux = total_flux1 + total_flux2 * 6
#     total_flux = total_flux1 + total_flux2 + total_flux3 + total_flux4 + total_flux5 + total_flux6 + total_flux7

    # Calculate the power fraction
    power_frac1 = total_flux1 / total_flux
    power_frac2 = total_flux2 / total_flux
#     power_frac3 = total_flux3 / total_flux
#     power_frac4 = total_flux4 / total_flux
#     power_frac5 = total_flux5 / total_flux
#     power_frac6 = total_flux6 / total_flux
#     power_frac7 = total_flux7 / total_flux
    
    flux1 /= total_flux1
    flux2 /= total_flux2
#     flux3 /= total_flux3
#     flux4 /= total_flux4
#     flux5 /= total_flux5
#     flux6 /= total_flux6
#     flux7 /= total_flux7

    # Assign other fluxes with the same values as flux2
    flux3 = flux2
    flux4 = flux2
    flux5 = flux2
    flux6 = flux2
    flux7 = flux2

    # Stack the fluxes
    flux = np.vstack((flux1, flux2, flux3, flux4, flux5, flux6, flux7))

    # Store flux data for the current time value
    flux_dict[i] = flux
    power_frac_dict[i] = np.array([power_frac1, power_frac2, power_frac2, power_frac2, power_frac2, power_frac2, power_frac2])

power[0] = power_end
precursors[0] = precursor_end
T_salt[0] = T_salt_end
T_graph[0] = T_graph_end
reactivity[0] = reactivity_end
beta_lost[0] = beta_lost_end

# Initialize matrices for precursor and temperatures
A = np.zeros((n_elements, n_elements)) 
C = np.zeros((n_elements, n_elements))
E = np.zeros((n_elements, r_elements-1, r_elements-1))
F = np.zeros((r_elements-1))
sums = np.zeros(6)

# Time loop
start_time = time.time()

for t in range(len(t_values)-1):
    if t in flux_t_list:
        flux = flux_dict[t]
        power_frac = power_frac_dict[t]
        
    for n in range(number_segments):
        print(f'Iteration: {t}', end='\r')  # Print the current iteration number and overwrite it on the next iteration
        
        C[indices, indices] = a_j_temp_s(indices, t, n)
        C[indices, indices - 1] = b_j_temp_s(indices, t, n)
        D = B_j_s(t,n)

        T_salt[t + 1, n, :] = np.linalg.solve(C, D)

        for i in range(n_elements):
            if graphite_present(i):
                j = np.arange(r_elements-1)
                E[i, j, j] = a_j_temp_g(i, j)
                E[i, j[:-1], j[:-1] + 1] = b_j_temp_g(i, j[:-1])
                E[i, j, j - 1] = c_j_temp_g(i, j)
                F = B_j_g(t, i, j, n)
                T_graph[t + 1, n, i, :] = np.linalg.solve(E[i, :, :], F)
            else:
                T_graph[t + 1, n, i, :] = np.zeros(r_elements-1)

        reactivity[t+1, n] = alpha[n] * (np.sum(flux[n] * T_salt[t+1,n,:]) - T_ref[n]) 
        
        for g in range(6):
            A[indices, indices] = a_j_precursor(indices, g, t, n)
            A[indices, indices - 1] = b_j_precursor(indices, t, n)
            B = B_j_precursor(g, t,n)

            precursors[t+1,n,:,g] = np.linalg.solve(A, B)        
    
    for g in range(6):
        sums[g] = lambda_groups[g] * np.sum(surface_salt_values * dz * np.sum(precursors[t+1,:,:,g], axis = 0) *1/np.sum(power_frac) * adjoint_flux_flat)# / np.sum(flux[n] * adjoint_flux_flat)
    beta_lost[t+1] = total_beta_eff - Lambda / (power[t]) * np.sum(sums)
    first_term = (1 / dt - (beta_lost[0] - total_beta_eff) / Lambda)**-1
    second_term = power[t] / dt + np.sum(sums)
    power[t+1] = first_term * second_term
    
end_time = time.time()
elapsed_time = end_time - start_time
print(f'The time it took to run {t} iterations was: {elapsed_time/60:.2f} minutes')

In [ ]:
number = -1
Segment = 0
plt.plot(t_values[:number], power[:number])
# plt.title('Power over time')
plt.xlabel('Time [s]')
plt.ylabel('Power')
# plt.grid()
# plt.savefig(r'Figures\Power over time 2flow', dpi = 300)
plt.show()

z_core = np.linspace(0, dz * (n_up + n_plenum + n_down), (n_up + n_plenum + n_down))
z_whole = np.linspace(0, dz * n_elements, n_elements)
graphtime = 10
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 0], label = 'Group 1')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 1], label = 'Group 2')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 2], label = 'Group 3')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 3], label = 'Group 4')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 4], label = 'Group 5')
# plt.plot(z_core, precursors[graphtime,Segment,:(n_up + n_plenum + n_down), 5], label = 'Group 6')
# plt.legend()
# plt.title(f'Precursor groups over core at t = {graphtime*dt}')
# plt.xlabel('z (cm)')
# plt.ylabel('Number of precursors')
# plt.grid()
# plt.show()

# plt.plot(z_whole, precursors[graphtime,Segment,:, 0], label = 'Group 1')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 1], label = 'Group 2')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 2], label = 'Group 3')
# plt.plot(z_whole, precursors[graphtime,Segme nt,:, 3], label = 'Group 4')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 4], label = 'Group 5')
# plt.plot(z_whole, precursors[graphtime,Segment,:, 5], label = 'Group 6')
# plt.legend()
# plt.title(f'Precursor groups over everything at t = {graphtime*dt}')
# plt.xlabel('z (cm)')
# plt.ylabel('Number of precursors')
# plt.grid()
# plt.show()

# plt.plot(t_values, precursors[:,Segment,0,0], label = 'Group 1')
# plt.plot(t_values, precursors[:,Segment,0,1], label = 'Group 2')
# plt.plot(t_values, precursors[:,Segment,0,2], label = 'Group 3')
# plt.plot(t_values, precursors[:,Segment,0,3], label = 'Group 4')
# plt.plot(t_values, precursors[:,Segment,0,4], label = 'Group 5')
# plt.plot(t_values, precursors[:,Segment,1,5], label = 'Group 6')
# plt.title('Precursor groups over time in first element')
# plt.xlabel('Time (s)')
# plt.ylabel('Number of precursors')
# plt.grid()
# plt.legend()
# plt.show()

plt.plot(t_values[:number], reactivity[:number, Segment])
# plt.title('Temperature reactivity over time')
plt.xlabel('Time [s]')
plt.ylabel('Reactivity')
# plt.grid()
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
# plt.savefig(r'Figures\Reactivity over time 2flow2', bbox_inches='tight', dpi = 300)
plt.show()

plt.plot(t_values[:], beta_lost[:])
# plt.title('Eq reactivity over time')
plt.xlabel('Time [s]')
plt.ylabel('Reactivity')
# plt.grid()
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
# plt.savefig(r'Figures\eq Reactivity over time 2flow2', bbox_inches='tight', dpi = 300)
plt.show()

# plt.plot(t_values, beta_lost+np.sum(reactivity, axis = 1))
# # plt.title('Total reactivity over time')
# plt.xlabel('Time [s]')
# plt.ylabel('Reactivity')
# # plt.grid()
# plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
# plt.savefig(r'Figures\total Reactivity over time 2flow2', bbox_inches='tight', dpi = 300)
# plt.show()

plt.plot(t_values[:number], np.mean(T_salt[:number,Segment], axis = 1))
# plt.title('Salt temperature over time')
plt.xlabel('Time [s]')
plt.ylabel('Temperature [K]')
# plt.grid()
# plt.savefig(r'Figures\Ts over time 2flow2', dpi = 300)
plt.show()

plt.plot(z_whole, T_salt[-1,Segment])
# plt.title('Salt temperature over reactor')
plt.xlabel('z [cm])')
plt.ylabel('Temperature [K]')
# plt.grid()
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
plt.tight_layout()
# plt.savefig(r'Figures\Ts over reactor 2flow2', bbox_inches='tight', dpi = 300)
plt.show()

plt.plot(t_values[:number], np.mean(T_graph[:number,Segment,n_up+n_plenum:core_elements,1], axis = 1))
# plt.title('Graphite temperature over time')
plt.xlabel('Time [s]')
plt.ylabel('Temperature [K]')
# plt.grid()
# plt.savefig(r'Figures\Tg over time 2flow2', dpi = 300)
plt.show()

plt.plot(z_core, np.mean(T_graph[-1,Segment,:core_elements,:], axis = 1))
# plt.title('Graphite temperature over core')
plt.xlabel('z [cm]')
plt.ylabel('Temperature [K]')
# plt.grid()
# plt.savefig(r'Figures\Tg over core 2flow2', dpi = 300)
plt.show()

plt.plot(r(50)[:-1], T_graph[-1,Segment,50,:])
# plt.title('Graphite temperature as function of r')
plt.xlabel('Radius [cm]')
plt.ylabel('Temperature [K]')
# plt.grid()
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
# plt.savefig(r'Figures\Tg over r 2flow2', dpi = 300)
plt.show()

In [ ]:
Segment = 0

power_in_megawatts2 = (g_zt(t)[-1,Segment] * rho_cp_fuel * (np.max(T_salt[-1,Segment]) - np.min(T_salt[-1,Segment]))) * 1e-6
power_in_megawatts3 = power_frac[Segment]*power[-1]*1e-6

print(f'The power according to thermal hydraulics is: {power_in_megawatts2:.2f} MW')
print(f'The power according to the neutronics is: {power_in_megawatts3:.2f} MW')
print(f'The mean salt temperature at the end is {np.mean(T_salt[-1,Segment]):.3f} K')
a = T_graph[-1,Segment,:,0]
a = a[a != 0]
print(f'The mean graphite temperature at the end is {np.mean(a):.3f} K')

# print(f'The time it takes to flow through the whole reactor is: {np.sum(surface_salt_values*dz)/g_zt(t):.2f} s')
print(f'The difference between the power from neutronics and thermal hydraulics is: {(power_in_megawatts3 - power_in_megawatts2)/power_in_megawatts2 *100} %')

# temp_diff_up = np.mean(frac(i)[0] * flux_flat[:n_up]/dz * power[-1] * 1 /(surface(10,j)[0] * heat_coef_values[:n_up]))
# temp_diff_down = np.mean(frac(i)[0] * flux_flat[(n_up+n_plenum):(n_up+n_plenum+n_down)]/dz * power[-1] * 1 /(surface(100,j)[0] * heat_coef_values[(n_up+n_plenum):(n_up+n_plenum+n_down)]))
# print(f'The temperature difference up should be: {temp_diff_up}')
# print(f'The temperature difference up is: {-np.mean(T_salt[-1,:n_up] - T_graph[-1,:n_up,0])}')
# print(f'The temperature difference down should be: {temp_diff_down}')
# print(f'The temperature difference down is: {-np.mean(T_salt[-1,(n_up+n_plenum):(n_up+n_plenum+n_down)] - T_graph[-1,(n_up+n_plenum):(n_up+n_plenum+n_down),0])}')

number = 100
print(f'Fraction of power is: {frac(number)[number] * flux[Segment,number]/dz * power[-1]*power_frac[Segment]}')
print(f'Heat flux is: {channel_values[number] / dz * heat_coef_values[number] * surface(number,j)[0] * (T_graph[-1,Segment,number,0] - T_salt[-1,Segment,number])}')

Power_up = np.sum(flux[Segment,:n_up]) * power[-1]*1e-6 *power_frac[Segment]
Power_down = np.sum(flux[Segment,n_up+n_plenum:n_up+n_plenum+n_down]) * power[-1]*1e-6*power_frac[Segment]

T_up = g_zt_values*rho_cp_fuel*(T_salt[-1,Segment,n_up-1]-T_salt[-1,Segment,-1])*1e-6
T_down = g_zt_values*rho_cp_fuel*(T_salt[-1,Segment,n_up+n_plenum+n_down+1]-T_salt[-1,Segment,n_up+n_plenum-1])*1e-6
# print(f'The power up is: {Power_up:.2f}')
# print(f'The temperature increase power up is: {T_up:.2f}')

# print(f'The power down is: {Power_down:.2f}')
# print(f'The temperature increase power down is: {T_down:.2f}')

print(f'The total power is: {power[-1]*1e-6:.2f} MW')
print(f'The power in the middle segment is {power_frac[0]*power[-1]*1e-6:.2f} MW and in the outer segment is {power_frac[1]*power[-1]*1e-6:.2f} MW')

In [ ]:
print(np.mean(T_salt[100,0,:core_elements]))
print(np.mean(T_salt[100,1,:core_elements]))
# print(np.mean(T_salt[50,2,:core_elements]))
# print(np.mean(T_salt[50,3,:core_elements]))
# print(np.mean(T_salt[50,4,:core_elements]))
# print(np.mean(T_salt[50,5,:core_elements]))
# print(np.mean(T_salt[50,6,:core_elements]))

a = T_graph[100,0,:,:]
a = a[a != 0]
print(f'The mean graphite temperature at the end is {np.mean(a):.3f} K')

b = T_graph[100,1,:,:]
b = b[b != 0]
print(f'The mean graphite temperature at the end is {np.mean(b):.3f} K')

# b = T_graph[50,2,:,:]
# b = b[b != 0]
# print(f'The mean graphite temperature at the end is {np.mean(b):.3f} K')

# b = T_graph[50,3,:,:]
# b = b[b != 0]
# print(f'The mean graphite temperature at the end is {np.mean(b):.3f} K')

# b = T_graph[50,4,:,:]
# b = b[b != 0]
# print(f'The mean graphite temperature at the end is {np.mean(b):.3f} K')

# b = T_graph[50,5,:,:]
# b = b[b != 0]
# print(f'The mean graphite temperature at the end is {np.mean(b):.3f} K')

# b = T_graph[50,6,:,:]
# b = b[b != 0]
# print(f'The mean graphite temperature at the end is {np.mean(b):.3f} K')

# np.sum(surface_salt_values[:core_elements]*dz/g_zt_values[0])